In [ ]:
import numpy as np
import scipy
from scipy.stats import norm

Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины X∼Nμ,σ2 лежит в интервале μ±c⋅σ. Чему равно точное значение константы c? Округлите ответ до четырёх знаков после десятичной точки. 

In [55]:
round(norm.ppf(0.9985),4)

2.9677

В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

In [37]:
n1 = 11037
m1=104
p1 = float(m1)/n1
print p1

n2 = 11034
m2 = 189
p2 = float(m2)/n2
print p2

print round(p2-p1,4)

0.00942285041225
0.0171288743883
0.0077


Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки. 

$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [26]:
def proportions_confint_diff_ind(p1, n1, p2, n2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
    return (left_boundary, right_boundary)

In [51]:
print np.round_(proportions_confint_diff_ind(p2, n2, p1, n1),4)

[ 0.0047  0.0107]


Продолжим анализировать данные эксперимента Гарвардской медицинской школы.

Для бернуллиевских случайных величин X∼Ber(p) часто вычисляют величину $\frac{p}{1−p}$, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо $p$ нужно подставить $\hat{p}$. Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как

189110341−18911034=18911034−189≈0.0174

Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки. 

In [29]:
def odds(p):
    return p/(1 - p)

In [32]:
odds_asp = odds(p1)
odds_plac = odds(p2)

print odds_asp
print odds_plac
print round(odds_plac/odds_asp, 4)

0.00951248513674
0.0174273858921
1.8321


Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

    составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
    установите random seed=0;
    сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [42]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def get_odds_relation(samples1, samples2):
    p1 = sum(samples1)/float(len(samples1))
    p2 = sum(samples2)/float(len(samples2))
    return odds(p2)/odds(p1)    

In [44]:
samples1 = np.array([1]*m1 + [0]*(n1 - m1))
samples2 = np.array([1]*m2 + [0]*(n2 - m2))

np.random.seed(0)
odds_rel = map(get_odds_relation, get_bootstrap_samples(samples1, 1000), get_bootstrap_samples(samples2, 1000))

In [50]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

print "95% confidence interval:",  map(lambda x: round(x,5), stat_intervals(odds_rel, 0.05))

95% confidence interval: [1.44419, 2.34321]
